<a href="https://colab.research.google.com/github/rikoder/Large_Language_Model_Projects/blob/main/LLM4BI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install --upgrade openai
!pip install python-dotenv
!pip install tiktoken
!pip install gspread pandas
!pip install gspread gspread_dataframe pandas
%pip install --upgrade gspread #IMPORTANT
!pip install pandasai

In [ ]:
import pandas as pd
import numpy as np
import re
import os
import openai
import pandas as pd
import tiktoken # https://github.com/openai/tiktoken
import numpy as np
import itertools
import string
import random
import time
import datetime
from datetime import datetime, timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from google.colab import drive
drive.mount('/content/drive')
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import auth
import warnings
warnings.filterwarnings("ignore")
from pandasai import PandasAI
from pandasai.llm.openai import OpenAI
from sklearn import linear_model
import warnings
warnings.filterwarnings("ignore")

Mounted at /content/drive


In [ ]:
openai.api_key = "API KEY"
api_key ="API KEY"

**Functions for Supply Chain Metrics and KPIs and OpenAI API call**

In [ ]:
def calculate_ABC_classification(product_master, sale_data, period_months=12, A=0.7, B=0.9, C=1):
  from datetime import datetime
  from dateutil.relativedelta import relativedelta
  # calculate the net invoice amount for each item for the specified period of analysis
  date_formatted = (datetime.now() - relativedelta(months=period_months)).strftime('%Y-%m-%d')
  product_sales = sale_data[sale_data['sale_order_date'] >= date_formatted].groupby('product_id')['net_invoice_amount'].sum().reset_index()
  # Calculate the net invoice amount for each item
  product_sales = product_sales.groupby('product_id')['net_invoice_amount'].sum().reset_index()
  ##complete the list of products from product master as some product may be missing in product_sales
  complete_product_sales = pd.merge(product_master[['product_id']], product_sales, on='product_id', how='left')
  complete_product_sales['net_invoice_amount'].fillna(0, inplace=True)
  # Sort the sales in descending order
  complete_product_sales = complete_product_sales.sort_values(by='net_invoice_amount', ascending=False)
  complete_product_sales['cum_invoice_amount'] = complete_product_sales['net_invoice_amount'].cumsum()
  complete_product_sales['cum_sale_contribution'] = complete_product_sales['cum_invoice_amount'] / complete_product_sales['net_invoice_amount'].sum()
  condition_A = (complete_product_sales['cum_sale_contribution'] <= A)
  condition_B = (complete_product_sales['cum_sale_contribution'] > A) & (complete_product_sales['cum_sale_contribution'] <= B)
  condition_C = (complete_product_sales['cum_sale_contribution'] > B) & (complete_product_sales['cum_sale_contribution'] <= C)
  complete_product_sales['ABC_class'] = np.select([condition_A, condition_B, condition_C], ['A', 'B', 'C'], default='C')
  # merge ABC classification back to product_master and fill missing values with 'C'
  product_master = pd.merge(product_master, complete_product_sales[['product_id', 'ABC_class']], on='product_id', how='left')
  product_master['ABC_class'].fillna('C', inplace=True)
  sale_data = pd.merge(sale_data, product_master[['product_id', 'ABC_class']], on='product_id', how='left')
  return product_master, sale_data

def calculate_fill_rate(filtered_sale_data):
    # calculating total order value
    filtered_sale_data['total_order_value'] = filtered_sale_data['sale_order_qty'] * filtered_sale_data['price']
    total_order_value = filtered_sale_data['total_order_value'].sum()
    net_invoice_amount= filtered_sale_data['net_invoice_amount'].sum()
    fill_rate = 100*net_invoice_amount/total_order_value
    return fill_rate

def read_data():
  product_master = pd.read_csv('/content/drive/My Drive/supply_chain_gpt/product_master.csv')
  sale_data = pd.read_csv('/content/drive/My Drive/supply_chain_gpt/sale_data.csv')
  SOH_history = pd.read_csv('/content/drive/My Drive/supply_chain_gpt/SOH_history.csv')
  distribution_master= pd.read_csv('/content/drive/My Drive/supply_chain_gpt/distribution_master.csv')
  date_cols = [col for col in sale_data.columns if re.search('date', col, re.IGNORECASE)]
  sale_data[date_cols] = sale_data[date_cols].apply(pd.to_datetime)
  date_cols = [col for col in SOH_history.columns if re.search('date', col, re.IGNORECASE)]
  SOH_history[date_cols] = SOH_history[date_cols].apply(pd.to_datetime)
  return

def get_embedding(text):
    result = openai.Embedding.create(
      model='text-embedding-ada-002',
      input=text
    )
    return result["data"][0]["embedding"]

def vector_similarity(vec1,vec2):
  return np.dot(np.array(vec1), np.array(vec2))

def ask_gpt_3(question,scm_context):
    #scm_context['embedding'] = scm_context['key_words'].apply(get_embedding)
    prompt_embedding = get_embedding(question)
    scm_context["prompt_similarity"] = scm_context['embedding'].apply(lambda vector: vector_similarity(vector, prompt_embedding))
    scm_context =scm_context.sort_values(by ="prompt_similarity", ascending = False)
    top_context =scm_context[scm_context['key_words']!= 'Generic']
    top_context = top_context[:2]
    generic_row = scm_context[scm_context['key_words'] == 'Generic']
    top_context = generic_row.append(top_context)
    top_context = top_context['context'].tolist()
    top_context = '\n'.join(top_context)

    prompt = f"""{top_context}
              Question: give only python code using pandas. do not use python dictionaries.
              call functions and make use of function.do not use read_and_convert().
              do not give explainations starting with #, strictly use given context.
              product master does not have  any date column so strictly do not filter product master on any type of date
              strictly include python code current_date = pd.to_datetime('today')
              strictly use only above mentioned dataframes and columns without removing _ in their names.make sure to check all variables are defined.{question}
              Answer:"""

    openai.api_key =api_key
    prompt = (prompt)
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt= prompt,
        temperature=0,
        max_tokens=1000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )
    output = response["choices"][0]["text"]
    code = output
    lines = code.splitlines()
    filtered_lines = [line for line in lines if not (line.startswith("#") or ".csv" in line or "Error" in line or "error" in line or "assuming" in line or "use" in line)]

    lines = code.split("\n")
    new_lines = []
    for line in lines:
        line = line.replace("<code>", "")
        line = line.replace("</code>", "")
        new_lines.append(line)
    ### read all data###
    new_code = "\n".join(new_lines)
    product_master = pd.read_csv('/content/drive/My Drive/supply_chain_gpt/product_master.csv')
    sale_data = pd.read_csv('/content/drive/My Drive/supply_chain_gpt/sale_data.csv')
    SOH_history = pd.read_csv('/content/drive/My Drive/supply_chain_gpt/SOH_history.csv')
    distribution_master= pd.read_csv('/content/drive/My Drive/supply_chain_gpt/distribution_master.csv')
    date_cols = [col for col in sale_data.columns if re.search('date', col, re.IGNORECASE)]
    sale_data[date_cols] = sale_data[date_cols].apply(pd.to_datetime)
    date_cols = [col for col in SOH_history.columns if re.search('date', col, re.IGNORECASE)]
    SOH_history[date_cols] = SOH_history[date_cols].apply(pd.to_datetime)
    new_code = "\n".join(new_lines)
    ################################
    return print(new_code),print(exec(new_code))


**Read and Preprocess Suppply Chain Data**

In [ ]:
product_master = pd.read_csv('/content/drive/My Drive/supply_chain_gpt/product_master.csv')
sale_data = pd.read_csv('/content/drive/My Drive/supply_chain_gpt/sale_data.csv')
SOH_history = pd.read_csv('/content/drive/My Drive/supply_chain_gpt/SOH_history.csv')
distribution_master= pd.read_csv('/content/drive/My Drive/supply_chain_gpt/distribution_master.csv')
date_cols = [col for col in sale_data.columns if re.search('date', col, re.IGNORECASE)]
sale_data[date_cols] = sale_data[date_cols].apply(pd.to_datetime)
date_cols = [col for col in SOH_history.columns if re.search('date', col, re.IGNORECASE)]
SOH_history[date_cols] = SOH_history[date_cols].apply(pd.to_datetime)

**Read Supply Chain Context, Prompts and get embeddings for Input into Large Language Model(GPT-3) from Google Sheet**

In [ ]:
# Import the libraries
from google.colab import auth
import gspread
from google.auth import default
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
worksheet = gc.open('scm_domain_context').sheet1
#get_all_values gives a list of rows
rows = worksheet.get_all_values()
df = pd.DataFrame(rows)
df.columns = df.iloc[0]
df = df.iloc[1:]
scm_context = df

In [ ]:
scm_context['embedding'] = scm_context['key_words'].apply(get_embedding)

In [ ]:
question ="lets think step by step, first do ABC classification and then Compare the fill rate for the top 10 B class products for the last 6 months to the same period last year and calculate the percentage change. and print percentage change"
ask_gpt_3(question,scm_context)



# Import relativedelta from dateutil
from dateutil.relativedelta import relativedelta
# Get the current date
current_date = pd.to_datetime('today')
# Define the x period as 6 months
x = relativedelta(months=6)
# Calculate the start and end dates for x period ago
start_date_x = current_date - x
end_date_x = current_date
# Calculate the start and end dates for same period last year
start_date_ly = start_date_x - relativedelta(years=1)
end_date_ly = end_date_x - relativedelta(years=1)

# Calculate ABC classification
product_master, sale_data = calculate_ABC_classification(product_master, sale_data, period_months=18, A=0.7, B=0.9, C=1)

# Filter sale_data for last 6 months
sale_data_x = sale_data[(sale_data['sale_order_date']>=start_date_x) & (sale_data['sale_order_date']<=end_date_x)]
# Filter sale_data for same period last year
sale_data_ly = sale_data[(sale_data['sale_order_date']>=start_date_ly) & (sale_data['sale_order_date']<=end_date_ly)]

# Filter sale_data for B class products
s

(None, None)

In [ ]:
question_list = '''
what are current year to date revenues for DC DC-001. compare current year to date revenues with last year to date revenues for DC-001 and print the comparision
lets think step by step, first do ABC classification and then Compare the fill rate for the top 10 B class products for the last 6 months to the same period last year and calculate the percentage change. and print percentage change.
'''